In [1]:
import nltk
import random

# !pip install sentence_transformers
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

from elasticsearch import Elasticsearch
import pandas as pd

import torch
import torch.nn.functional as F

import time

from transformers import AutoModelForSequenceClassification, AutoTokenizer
model_class = AutoModelForSequenceClassification.from_pretrained('morenolq/spotify-podcast-advertising-classification')
tokenizer = AutoTokenizer.from_pretrained('morenolq/spotify-podcast-advertising-classification')

# model = SentenceTransformer('pinecone/distiluse-podcast-nq')

# Only run this once, they will be downloaded.
nltk.download('stopwords',quiet=True)
nltk.download('wordnet',quiet=True)
nltk.download('punkt',quiet=True)
nltk.download('omw-1.4',quiet=True)

Downloading: 100%|██████████| 748/748 [00:00<00:00, 249kB/s]
Downloading: 100%|██████████| 433M/433M [00:23<00:00, 18.2MB/s] 
Downloading: 100%|██████████| 213k/213k [00:00<00:00, 10.7MB/s]
Downloading: 100%|██████████| 669k/669k [00:00<00:00, 4.46MB/s]
Downloading: 100%|██████████| 343/343 [00:00<00:00, 57.3kB/s]


True

In [3]:
results = pd.read_csv("output/test_transcripts.csv")
test_1_transcript = results.loc[1, 'transcript']
sent_tokens = nltk.sent_tokenize(test_1_transcript)# converts to list of sentences
len(sent_tokens)

676

In [4]:
sentence_outputs_tuples = []
start = time.time()
for i, s in enumerate(sent_tokens[:200]): 
    if i==0:
        context = "__START__"
    else:
        context = sent_tokens[i-1] 
    out = tokenizer(context,
                    s,
                    padding = "max_length",
                    max_length = 256,
                    truncation=True,
                    return_attention_mask=True,
                    return_tensors = 'pt')
    outputs = model_class(**out)
    
    # probabilities = F.softmax(outputs.logits, dim=1)
    predictions = torch.argmax(outputs.logits, dim=-1)

    if predictions.item() == 1:
        sentence_outputs_tuples.append((s, predictions.item()))

end = time.time()
print("Time taken:", (end-start)/60, "minutes")


KeyboardInterrupt: 

In [8]:
test = []
len(test)

0

In [5]:
for sentences in sentence_outputs_tuples:
    if sentences[1] == 1:
        print(sentences[0])

[("What's up, everybody?", 0),
 ('Welcome to the in the dome podcast podcast body are you doing hey, how you doing?',
  0),
 ("I'm pretty good myself.", 0),
 ('All right.', 0),
 ('What do you want to talk about today?', 0),
 ('We got a breakdown.', 0),
 ("I know there's not much stock boat.", 0),
 ("We still haven't broken down the Columbus game, but cardiac / comeback kids come back.",
  0),
 ('You know what that game room.', 0),
 ('Okay, I believe off the where we thought we were talking about a couple of episodes ago were talking about house like a toxic relationship and I was starting to get sucked back in.',
  0),
 ('Yep.', 0),
 ("And then I think you were to it's like when fully back on board is fully back with the Calgary Flames right now to start that podcast.",
  0),
 ("I was like, I'm not I'm not falling for it.", 0),
 ("I've been hurt too many times by these guys, but by the end of that podcast we switched but I was still whatever.",
  0),
 ("Yeah, I'm fine.", 0),
 ("I'm tol

In [38]:
test_transcript_class = pd.DataFrame.from_records(sentence_outputs_tuples)

In [42]:
test_transcript_class.sort_values(by=1, ascending=False).head(10)[0].values

array(['No, I believe I do believe I will say that.', 'I like the wild.',
       'But for it was fun.', "Okay, I don't have it in front of me.",
       "You can't follow that up with a loss you continue that momentum with another win.",
       "What's up, everybody?",
       "And again like we mentioned this yesterday to it's like why Riddick had to Riddick was the one who played the majority of those games.",
       'The guy that is questioned about how much of a workload can handle and the past.',
       "He like you're just playing night after night after night.",
       "And then even when Talbot starts to like play Lights Out you still start going back to Rick like the I don't like  I feel like more than anything that's fucked with his game."],
      dtype=object)

In [43]:
test_transcript_class.to_csv("output/test_transcript_class.csv", index=False)